<a href="https://colab.research.google.com/github/KunaCornejo/SmartGatewaySelection/blob/main/SmartSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import genfromtxt
from geopy.distance import great_circle

In [2]:
#import git
!git clone https://github.com/KunaCornejo/SmartGatewaySelection.git


Cloning into 'SmartGatewaySelection'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [32]:
ls 'SmartGatewaySelection'

c_n_ul.csv  SmartSelection.ipynb


In [24]:
c_n_ul = genfromtxt('/content/SmartGatewaySelection/c_n_ul.csv', delimiter=',') 
c_n_ul=list(c_n_ul)

In [30]:
ubicaciones=['Panama','San Jose','Tegucigalpa','Mexico City','Monterrey','Guadalajara'\
             ,'Tijuana','La Habana','Sto. Domingo','San Salvador','San Juan',\
             'Torreon','La Paz (BC)','Veracruz','Cancun','Queretaro'\
             ,'Tuxtla (Gtz)','Cd. Juarez','San Pedro Sula','Belmopan','Kingston'\
             ,'Merida','Pto. Cabezas','Oaxaca']
ubicaciones=np.transpose([ubicaciones])
C_N_UL=np.array([c_n_ul],dtype=object) 
C_N_UL=np.transpose(C_N_UL)
C_N_UL=np.delete(C_N_UL, 15, 0) #Get rid of Chihuahua, it is not necessary
CN_UP=np.concatenate((ubicaciones,C_N_UL),axis=1) #Concatenate two lists
print(CN_UP)

[['Panama' 32.4401636]
 ['San Jose' 32.4611016]
 ['Tegucigalpa' 32.4450513]
 ['Mexico City' 32.3918091]
 ['Monterrey' 32.313151]
 ['Guadalajara' 32.3588726]
 ['Tijuana' 32.0981607]
 ['La Habana' 32.3380601]
 ['Sto. Domingo' 32.2887155]
 ['San Salvador' 32.4521009]
 ['San Juan' 32.2438352]
 ['Torreon' 32.301041]
 ['La Paz (BC)' 32.2705558]
 ['Veracruz' 32.4018877]
 ['Cancun' 32.3779865]
 ['Queretaro' 32.3736176]
 ['Tuxtla (Gtz)' 32.4294244]
 ['Cd. Juarez' 32.195777]
 ['San Pedro Sula' 32.4354086]
 ['Belmopan' 32.4215586]
 ['Kingston' 32.3572017]
 ['Merida' 32.3851709]
 ['Pto. Cabezas' 32.4314209]
 ['Oaxaca' 32.4205395]]


In [31]:
# Python code to sort the array using second element  
# of sublist Function to sort using sorted() 
def Sort(sub_li): 
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of  
    # sublist lambda has been used 
    return(sorted(sub_li, key = lambda x: x[1], reverse = True)) 
    #X[] es la columna que vamos a ordenar
# Driver Code 
sub_li = CN_UP
sub_li= np.array(Sort(sub_li))  #Just test the sorted list 
print(sub_li)

[['San Jose' 32.4611016]
 ['San Salvador' 32.4521009]
 ['Tegucigalpa' 32.4450513]
 ['Panama' 32.4401636]
 ['San Pedro Sula' 32.4354086]
 ['Pto. Cabezas' 32.4314209]
 ['Tuxtla (Gtz)' 32.4294244]
 ['Belmopan' 32.4215586]
 ['Oaxaca' 32.4205395]
 ['Veracruz' 32.4018877]
 ['Mexico City' 32.3918091]
 ['Merida' 32.3851709]
 ['Cancun' 32.3779865]
 ['Queretaro' 32.3736176]
 ['Guadalajara' 32.3588726]
 ['Kingston' 32.3572017]
 ['La Habana' 32.3380601]
 ['Monterrey' 32.313151]
 ['Torreon' 32.301041]
 ['Sto. Domingo' 32.2887155]
 ['La Paz (BC)' 32.2705558]
 ['San Juan' 32.2438352]
 ['Cd. Juarez' 32.195777]
 ['Tijuana' 32.0981607]]


In [33]:
#Unzip the .csv file
import zipfile #import ZipFile
with zipfile.ZipFile('/content/SmartGatewaySelection/Matriz_Train_90.csv.zip','r') as zip_ref:
    zip_ref.extractall('content')

FileNotFoundError: ignored

In [34]:
#Se importa la matriz de atenuacion por lluvia obtenido de la ITU-R 1853-1
#Matriz_Arain.csv son los datos obtenidos de la ITU-R 1853-1 de cada locación
#Matriz_Train.csv son los datos previamente entrenados de las locaciones en orden de 'ubicaciones'
Matrix_Arain = genfromtxt('/content/Matriz_Train_90.csv', delimiter=',')
Matrix_Arain = np.delete(Matrix_Arain, 15, 0) #Hay que eliminar una fila, la 15 y Chihuahua
#Matriz_Arain=np.transpose(np.array(Matriz_Arain,dtype=object)) #Matriz_Train.csv no necesita ser transpuesta

OSError: ignored